In [3]:
import pandas as pd
import os
from datetime import datetime


In [4]:
ndmi_dir = 'data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024'

In [5]:
df = pd.read_csv('openweathermap_data.csv')
df['precipitation_value'] = df['precipitation'].str.extract(r"'total': ([\d.]+)").astype(float)

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def calculate_precipitation_sums(df, tiff_folder):
    # Get list of TIFF files
    tiff_files = [f for f in os.listdir(tiff_folder) if f.endswith('.tif')]
    
    results = []
    
    for tiff_file in tiff_files:
        # Extract dates from filename
        dates = tiff_file.replace('NDMI_', '').replace('.tif', '').split('_to_')
        start_date = datetime.strptime(dates[0], '%Y-%m-%d')
        end_date = datetime.strptime(dates[1], '%Y-%m-%d')
        
        # Get precipitation sum for date range
        mask = (df.date >= start_date) & (df.date <= end_date)
        precip_sum = df.loc[mask, 'precipitation_value'].sum()
        
        results.append({
            'tiff_file': tiff_file,
            'start_date': start_date,
            'end_date': end_date,
            'precipitation_sum': precip_sum
        })
    
    # Create new dataframe with results
    result_df = pd.DataFrame(results)
    return result_df

In [8]:
result_df = calculate_precipitation_sums(df, ndmi_dir)
result_df.to_csv('precipitation_sums.csv', index=False)

In [9]:
result_df.head()

,tiff_file,start_date,end_date,precipitation_sum
0,NDMI_2019-01-01_to_2019-01-08.tif,2019-01-01,2019-01-08,0.00
1,NDMI_2019-01-15_to_2019-01-22.tif,2019-01-15,2019-01-22,5.24
2,NDMI_2019-02-05_to_2019-02-12.tif,2019-02-05,2019-02-12,3.60
3,NDMI_2019-02-12_to_2019-02-19.tif,2019-02-12,2019-02-19,0.57
4,NDMI_2019-02-19_to_2019-02-26.tif,2019-02-19,2019-02-26,31.26
